# 자동차 이미지들을 학습 및 테스트합니다.

훈련 데이터는 각 240개씩 총 960개로 구성되어있고,
시험 데이터는 각 60개씩 총 240개로 구성되어있습니다.

차종은 세단, 스포츠카, SUV, 픽업트럭으로 네가지를 구분하였고 종류별 숫자 코드는 다음과 같습니다.
0 = sedan, 1 = sportscar, 2 = suv, 3 = truck

In [1]:
import os, random
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image

Instructions for updating:
non-resource variables are not supported in the long term


    Namin Neural Network
    -----------------
    conv1 - relu1 - pool1 - bn1
    conv2 - relu2 - pool2 - bn2
    conv3 - relu3 - pool3 - bn3
    affine - softmax
    
    Parameters
    ----------
    Input Layer(Input Size): 100*100*3
    First Layer: Conv1(3EA, 3*3*3, Strides=3, Padding=1) - ReLU1
                - 34*34*3 - Pool1(2*2, Strides=1) - 33*33*3 - Bn1
    Second Layer: Conv2(6EA, 6*6*3, Strides=1, Padding=VALID) - ReLU2
                - 28*28*6 - Pool2(2*2, Strides=2) - 14*14*6 - Bn2
    Third Layer: Conv3(9EA, 3*3*6, Strides=3, Padding=2) - ReLU3
                - 6*6*9 - Pool3(2*2, Strides=2) - 3*3*9 - Bn3
    Output Layer: Affine(W=3*3*9, B=9) - Output Nodes = 4(Sedan, Coupe, SUV, PickupTruck)

In [2]:
# Load to Sedan, Coupe, SUV, PickupTruck Images
trainlist, testlist = [], []
with open('train.txt') as f:
    for line in f:
        tmp = line.strip().split()
        trainlist.append([tmp[0], tmp[1]])
        
with open('test.txt') as f:
    for line in f:
        tmp = line.strip().split()
        testlist.append([tmp[0], tmp[1]])

In [3]:
# Image Preprocessing
IMG_H = 100
IMG_W = 100
IMG_C = 3

def readimg(path):
    img = plt.imread(path)
    return img

def batch(path, batch_size):
    img, label, paths = [], [], []
    for i in range(batch_size):
        img.append(readimg(path[0][0]))
        label.append(int(path[0][1]))
        path.append(path.pop(0))
        
    return img, label

In [4]:
# Neural Network
num_class = 4 # Sedan, Coupe, SUV, PickupTruck

with tf.Graph().as_default() as g:
    X = tf.placeholder(tf.float32, [None, IMG_H, IMG_W, IMG_C]) # Input Layer, X = tf.placeholder(tf.float32, [None, IMG_H, IMG_W, IMG_C])
    Y = tf.placeholder(tf.int32, [None]) # Y = tf.placeholder(tf.int32, [None])
    
    with tf.variable_scope('CNN'):
        # 1st Layer(Conv1 - relu1 - maxpool1 - bn1) = 33*33*3
        conv1 = tf.layers.conv2d(X, 3, 3, (3, 3), padding='SAME', activation=tf.nn.relu)
        pool1 = tf.layers.max_pooling2d(conv1, 2, (1, 1), padding='VALID')
        bn1 = tf.compat.v1.layers.batch_normalization(pool1, training=True)
        # 2nd Layer(Conv2 - relu2 - maxpool2 - bn2) = 14*14*6
        conv2 = tf.layers.conv2d(bn1, 6, 6, (1, 1), padding='VALID', activation=tf.nn.relu)
        pool2 = tf.layers.max_pooling2d(conv2, 2, (2, 2), padding='VALID')
        bn2 = tf.compat.v1.layers.batch_normalization(pool2, training=True)
        # 3rd Layer(Conv3 - relu3 - maxpool3 - bn3) = 3*3*9
        conv3 = tf.layers.conv2d(bn2, 9, 3, (3, 3), padding='SAME', activation=tf.nn.relu)
        pool3 = tf.layers.max_pooling2d(conv3, 2, (2, 2), padding='VALID')
        bn3 = tf.compat.v1.layers.batch_normalization(pool3, training=True)
        # Fully Connected Layer(Affine)
        affine1 = tf.layers.flatten(bn3)
        # Output Layer
        output = tf.layers.dense(affine1, num_class)
        
    # Softmax with Loss
    with tf.variable_scope('Loss'):
        Loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels= Y, logits=output))
    
    # Training with Adam    
    train_step = tf.train.AdamOptimizer(0.005).minimize(Loss) 
    saver = tf.train.Saver()
    
    tf.summary.scalar('Epoch-Loss', Loss)
    merged = tf.summary.merge_all()

Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.MaxPooling2D instead.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
Instructions for updating:
Use keras.layers.Flatten instead.
Instructions for updating:
Use keras.layers.Dense instead.


In [5]:
# Size
np.sum([np.product(var.shape) for var in g.get_collection('trainable_variables')]).value

1417

In [6]:
# Setting Batch with Training
batch_size = 1261
epoch = 1000

with tf.Session(graph=g) as sess:
    sess.run(tf.global_variables_initializer())
    summary_writer = tf.summary.FileWriter('tf_board', sess.graph)
    for i in range(epoch):
        batch_data, batch_label = batch(trainlist, batch_size)     
        _, loss, summary = sess.run([train_step, Loss, merged], feed_dict = {X: batch_data, Y: batch_label})
        print("Epoch:",i+1,"Loss:",loss)
        if i % 10 == 0:
            summary_writer.add_summary(summary, i)
            saver.save(sess, 'logs/model.ckpt', global_step = i+1)
        elif i+1 == epoch:
            summary_writer.add_summary(summary, i)
            saver.save(sess, 'logs/model.ckpt', global_step = i+1)

Epoch: 1 Loss: 1.8610898
Epoch: 2 Loss: 1.6784959
Epoch: 3 Loss: 1.5134746
Epoch: 4 Loss: 1.4753015
Epoch: 5 Loss: 1.4646184
Epoch: 6 Loss: 1.3870704
Epoch: 7 Loss: 1.3817285
Epoch: 8 Loss: 1.3844335
Epoch: 9 Loss: 1.3351525
Epoch: 10 Loss: 1.3237802
Epoch: 11 Loss: 1.3044883
Epoch: 12 Loss: 1.3024364
Epoch: 13 Loss: 1.2783118
Epoch: 14 Loss: 1.2546879
Epoch: 15 Loss: 1.263465
Epoch: 16 Loss: 1.2451932
Epoch: 17 Loss: 1.2223198
Epoch: 18 Loss: 1.2194121
Epoch: 19 Loss: 1.2012662
Epoch: 20 Loss: 1.2111149
Epoch: 21 Loss: 1.1874096
Epoch: 22 Loss: 1.180881
Epoch: 23 Loss: 1.1920776
Epoch: 24 Loss: 1.163473
Epoch: 25 Loss: 1.1649181
Epoch: 26 Loss: 1.1727526
Epoch: 27 Loss: 1.1476595
Epoch: 28 Loss: 1.1455699
Epoch: 29 Loss: 1.1407949
Epoch: 30 Loss: 1.1289343
Epoch: 31 Loss: 1.1253608
Epoch: 32 Loss: 1.1213796
Epoch: 33 Loss: 1.108736
Epoch: 34 Loss: 1.1028564
Epoch: 35 Loss: 1.0816852
Epoch: 36 Loss: 1.094297
Epoch: 37 Loss: 1.0782992
Epoch: 38 Loss: 1.0635791
Epoch: 39 Loss: 1.0692173


Epoch: 290 Loss: 0.041820917
Epoch: 291 Loss: 0.039877973
Epoch: 292 Loss: 0.042612877
Epoch: 293 Loss: 0.04115216
Epoch: 294 Loss: 0.040253565
Epoch: 295 Loss: 0.040685087
Epoch: 296 Loss: 0.043621987
Epoch: 297 Loss: 0.040378224
Epoch: 298 Loss: 0.037459224
Epoch: 299 Loss: 0.039730348
Epoch: 300 Loss: 0.039389238
Epoch: 301 Loss: 0.03638715
Epoch: 302 Loss: 0.035344873
Epoch: 303 Loss: 0.038259935
Epoch: 304 Loss: 0.035363477
Epoch: 305 Loss: 0.034990814
Epoch: 306 Loss: 0.035698023
Epoch: 307 Loss: 0.034423348
Epoch: 308 Loss: 0.0356228
Epoch: 309 Loss: 0.034822952
Epoch: 310 Loss: 0.033922404
Epoch: 311 Loss: 0.03472626
Epoch: 312 Loss: 0.03624654
Epoch: 313 Loss: 0.033759017
Epoch: 314 Loss: 0.032643322
Epoch: 315 Loss: 0.03556143
Epoch: 316 Loss: 0.03434453
Epoch: 317 Loss: 0.031596597
Epoch: 318 Loss: 0.03104816
Epoch: 319 Loss: 0.033347487
Epoch: 320 Loss: 0.029655647
Epoch: 321 Loss: 0.030191788
Epoch: 322 Loss: 0.02992104
Epoch: 323 Loss: 0.02950684
Epoch: 324 Loss: 0.030095

Epoch: 574 Loss: 0.0066273795
Epoch: 575 Loss: 0.0067479727
Epoch: 576 Loss: 0.006708878
Epoch: 577 Loss: 0.0065941513
Epoch: 578 Loss: 0.006520813
Epoch: 579 Loss: 0.0071959198
Epoch: 580 Loss: 0.0071055694
Epoch: 581 Loss: 0.006521168
Epoch: 582 Loss: 0.0070011513
Epoch: 583 Loss: 0.007492378
Epoch: 584 Loss: 0.0065689175
Epoch: 585 Loss: 0.0064419843
Epoch: 586 Loss: 0.0067782444
Epoch: 587 Loss: 0.006708307
Epoch: 588 Loss: 0.0063180067
Epoch: 589 Loss: 0.006121772
Epoch: 590 Loss: 0.0064362283
Epoch: 591 Loss: 0.006265606
Epoch: 592 Loss: 0.006265621
Epoch: 593 Loss: 0.0061097564
Epoch: 594 Loss: 0.0061297375
Epoch: 595 Loss: 0.006861941
Epoch: 596 Loss: 0.0064985366
Epoch: 597 Loss: 0.0060699256
Epoch: 598 Loss: 0.00683987
Epoch: 599 Loss: 0.00728442
Epoch: 600 Loss: 0.0061093657
Epoch: 601 Loss: 0.006148728
Epoch: 602 Loss: 0.0066691185
Epoch: 603 Loss: 0.0064414106
Epoch: 604 Loss: 0.005975826
Epoch: 605 Loss: 0.0057749464
Epoch: 606 Loss: 0.006121673
Epoch: 607 Loss: 0.0058514

Epoch: 851 Loss: 0.0032806955
Epoch: 852 Loss: 0.002733212
Epoch: 853 Loss: 0.0029095493
Epoch: 854 Loss: 0.003212214
Epoch: 855 Loss: 0.0028544292
Epoch: 856 Loss: 0.0027478007
Epoch: 857 Loss: 0.0026121284
Epoch: 858 Loss: 0.0028363806
Epoch: 859 Loss: 0.0027186922
Epoch: 860 Loss: 0.0026554943
Epoch: 861 Loss: 0.0025988596
Epoch: 862 Loss: 0.0026800174
Epoch: 863 Loss: 0.0027643433
Epoch: 864 Loss: 0.0026579718
Epoch: 865 Loss: 0.0025605496
Epoch: 866 Loss: 0.002993401
Epoch: 867 Loss: 0.0030099836
Epoch: 868 Loss: 0.0026188355
Epoch: 869 Loss: 0.0028590437
Epoch: 870 Loss: 0.0030949179
Epoch: 871 Loss: 0.0026830586
Epoch: 872 Loss: 0.002661589
Epoch: 873 Loss: 0.002636749
Epoch: 874 Loss: 0.0027091512
Epoch: 875 Loss: 0.0026103167
Epoch: 876 Loss: 0.0024767641
Epoch: 877 Loss: 0.002546498
Epoch: 878 Loss: 0.0025584372
Epoch: 879 Loss: 0.0025993786
Epoch: 880 Loss: 0.0025070934
Epoch: 881 Loss: 0.0025100738
Epoch: 882 Loss: 0.0028560273
Epoch: 883 Loss: 0.0027754554
Epoch: 884 Loss:

In [7]:
# Print an Accuracy
acc = 0

with tf.Session(graph=g) as sess:
    sess.run(tf.global_variables_initializer())
    checkpoint = tf.train.latest_checkpoint('logs')
    if checkpoint:
        saver.restore(sess, checkpoint)
    for i in range(len(testlist)):
        batch_data, batch_label = batch(testlist, 1)
        logit = sess.run(output, feed_dict = {X:batch_data})
        if np.argmax(logit[0]) == batch_label[0]:
            acc += 1
        else:
            print(logit[0], batch_label[0])
            
    print("Accuracy:", acc/len(testlist))

INFO:tensorflow:Restoring parameters from logs\model.ckpt-1000
[-17.278006    3.235529    6.8484416  -5.9314322] 0
[-10.057173    7.0168886   0.9351513  -4.7276454] 0
[-1.0380585 -4.0620213 -8.484668   3.5294194] 0
[  0.5391812   14.13793      0.63532877 -14.495421  ] 0
[-4.342651  -6.0655394 -0.7866046 13.003101 ] 0
[-17.934954  -15.7701025  11.571608    5.2379317] 0
[  2.9852672   -0.66685915 -13.561948    11.08674   ] 0
[  0.05917977  -9.059642     1.5708605  -12.91371   ] 0
[-6.996594  -0.7078459  6.398762  -3.585133 ] 0
[-1.9597287 -6.9169884 -3.3610299 -1.63502  ] 0
[-14.141863   -4.7289524   2.0295415   4.431017 ] 0
[  7.4803243 -11.335563    6.520249    9.691671 ] 0
[-2.2283869 -2.68149    9.259859  -7.708418 ] 0
[-2.4393115 12.857679   0.5217554 -5.489901 ] 0
[-23.165653    5.367371    2.1649225   2.6220615] 0
[ -0.66795397   4.922429     3.7625484  -11.746194  ] 0
[ -0.19961657  15.461643   -12.570179   -15.195143  ] 0
[-2.739008  -9.610336  -3.4827583  1.7071807] 0
[  2.2846